# RAG Sample

Building a RAG has 4 steps
- Load data
- Split data
- Vectorize and store data
- fetch data


In [ ]:
## env setup 

import os

os.environ.get("MISTRAL_API_KEY")

In [ ]:
## test the model
from langchain.chat_models import init_chat_model

llm = init_chat_model("mistral-large-latest", model_provider="mistralai")

## Loading
Langchain has many different types of loaders like PDFLoader, Markdown loader, YouTube loader, Web based loader etc


In [ ]:
## load PDF
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("./shekar_arukala.pdf")
pages = loader.load()

In [ ]:
print(len(pages))
page = pages[0]
print(page.page_content[0:900])

## Splitting
splotting the loaded data is the second step. There are multiple ways in which this can be done. 
Some popular splitters are 
- RecursiveCharacterTextSplitter
- CharacterTextSplitter


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter

In [ ]:
chunk_size =1000
chunk_overlap = 250
seperator = " "

In [ ]:
r_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, 
                                            chunk_overlap=chunk_overlap, 
                                            separators=['\n\n', '\n', ' ', ''])

c_splitter = CharacterTextSplitter(chunk_size=chunk_size, 
                                   chunk_overlap=chunk_overlap, 
                                   separator=' ')

In [ ]:
r_splits = r_splitter.split_documents(pages)
c_splits = c_splitter.split_documents(pages)

In [ ]:
print(len(r_splits))
print(len(c_splits))

## Embedding

In [ ]:
from langchain_mistralai import MistralAIEmbeddings

embedding = MistralAIEmbeddings(model="mistral-embed")

In [ ]:
### Testing embeddings

sentence1 = "i like dogs"
sentence2 = "i like canines"
sentence3 = "the weather is ugly outside"

embedding1 = embedding.embed_query(sentence1)
embedding2 = embedding.embed_query(sentence2)
embedding3 = embedding.embed_query(sentence3)

In [ ]:
import numpy as np

np.dot(embedding1, embedding2)
np.dot(embedding1, embedding3)
np.dot(embedding2, embedding3)

## Storage

In [ ]:
# pip install chromadb

from langchain.vectorstores import Chroma
# !rm -rf ./docs/chroma  # remove old database files if any

persist_directory = './chroma/'


In [ ]:
vectordb = Chroma.from_documents(
    documents=r_splits,
    embedding=embedding,
    persist_directory=persist_directory
)

In [ ]:
print(vectordb._collection.count())

In [ ]:
vectordb.persist()

## Retrieval

Retrieval is one of the key elements in an RAG. Similarity search is one of the most basic forms of retrieval. But it has some edge cases where it fails. WHen we want documents from specific source or when we want unique documents, it fails. 

Some popular Retrieval mechanisms are 

- Similarity search 
- Max Marginal Relevance
- Self Query Retriever
- Contextual Compression

In this exmple we are goinf to use Similarity search and MMR only as we have only one small source. 

In [ ]:
question = "what are your technical skills?"

docs_ss = vectordb.similarity_search(question,k=3)

docs_mmr = vectordb.max_marginal_relevance_search(question,fetch_k=3, k=2)

In [ ]:
len(docs_mmr)

In [ ]:
docs_mmr[0].page_content

## Q&A

In [ ]:
from langchain_mistralai.chat_models import ChatMistralAI

llm = ChatMistralAI(
    model_name='mistral-large-latest',
    temperature=0
)

In [ ]:
from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectordb.as_retriever()
)


In [ ]:
question = "what are your technical skills?"

In [ ]:

result = qa_chain.invoke({"query": question})


In [ ]:
result["result"]

## Q&A with prompt

In [ ]:
from langchain.prompts import PromptTemplate

# Build prompt
template = """Use the following pieces of context to answer the question at the end. 
              If you don't know the answer, just say that you don't know, don't try to make up an answer. 
              Use three sentences maximum. Keep the answer as concise as possible. 
              Always say "thanks for asking!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)


In [ ]:
from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectordb.as_retriever(search_type="mmr"),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)


In [ ]:
question = "what projects did you work on in Syntizen Technologies?"

In [ ]:
result = qa_chain({"query": question})

In [ ]:
result["result"]